<a href="https://colab.research.google.com/github/beloveddie/AI-Craft/blob/main/ResearchLink_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ResearchLink Recommendation using Comet's Opik for Tracking, Evaluation, and More

This notebook simply aims to present a project that inculcates the value of LLM Evaluation using Opik seamlessly.

By leveraging this tool from Comet we are able to track and see into the process flow of our LLM application.

Not just that, we can seamlessly compare the output of a given function, prompt or prompt template to others that are different for performance check and just to see which is outperforming which in regards to our set out metrics or end goal.

## Installation of required libraries

First off, we'll install the necessary libraries for this application

In [ ]:
!pip install -qU opik litellm requests PyPDF2 sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.9/285.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.3 MB/s eta 0:00:00


## Importation of required libraries

In [ ]:
import opik
from litellm import completion
import requests
import faiss
import numpy as np
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import xml.etree.ElementTree as ET
from google.colab import userdata
import os

Now we can go ahead and configure and set up the required variables and initialize the necessary `response` object.

You see, Opik is such an amazing tool, I'm sincerely loving it. It provides a package called `litellm`. `litellm` abstracts a way some complexities from us and gives us a standard interface to communicate with various LLM providers.

Amazing, right? 🤠

For this I'll be using my very favourite and go-to LLM provider - you should know by now... COHERE!

Cohere's models excel excellently at every task -from reasoning, to generation, to problem solving, to contextual tasks and the likes...

Let's go ahead, before I sell Cohere to you. Would be great though.

In [ ]:
# Configure Opik and Cohere for completion using litellm
opik.configure(api_key=userdata.get("COMET_API_KEY"))  # Replace with your Opik API key

## set ENV variables
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


Now we can access any LLMs from any providers of our choice by a simple and standard interface like so:

In [ ]:
# cohere call
response = completion(
    model="command-r",
    messages = [{ "content": "Hello, how are you?","role": "user"}]
)

Let's test this out 😜

In [ ]:
print(response.choices[0].message.content)

I'm doing well, thank you! I am an AI assistant chatbot, so I don't have emotions or feelings, but I am designed to be helpful and provide thorough responses to your queries. Is there anything I can help you with today?


Now let's define all our functions for this application.

We'll be decorating all of them with the `opik.track` decorator to make such we "track" the functions and their outputs respectively on our Comet's Opik Dashboard.

AHH, 😫 this is so sleek... Come on, think of "Explainable AI", "Presentable AI".

OPIK!! 😄😃 - a technology that gets me pumped up 😁💯

In [ ]:
@opik.track
def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file, and also extract the title of the paper.
    """
    pdf_reader = PdfReader(pdf_path)

    # Get the first page to extract the title (this can vary depending on PDF structure)
    first_page = pdf_reader.pages[0]
    text = first_page.extract_text()

    # Assuming the first line is the title of the paper (this can be customized based on document structure)
    title = text.split("\n")[0]  # Extract the first line as the title (adjust if needed)

    # Extract the rest of the text (excluding title)
    body_text = text[len(title):]

    # Return both the title and the body text
    return title, body_text

In [ ]:
@opik.track
def summarize_current_summary(document, instruction, current_summary, model="command-r"):
    """
    Generate a summary using Comet's command-r model with the Standardized litellm interface.
    """
    prompt = f"""
    Document: {document}
    Current summary: {current_summary}
    Instruction: {instruction}

    Revise the summary to make it more concise, technically accurate, and entity-dense, while aligning with the given instruction.

    """

    # Generate completion using command-r
    response = completion(
        model=model,  # Use command-r
        messages = [{ "content": prompt,"role": "user"}],
        max_token=1024,
    )

    return response.choices[0].message.content  # Extract the summary from the response

In [ ]:
@opik.track
def fetch_arxiv_papers(query, max_results=10):
    """
    Fetch research papers from ArXiv based on a search query.
    """
    base_url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "relevance",
        "sortOrder": "descending",
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        papers = []
        for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
            paper = {
                "title": entry.find("{http://www.w3.org/2005/Atom}title").text,
                "summary": entry.find("{http://www.w3.org/2005/Atom}summary").text,
                "authors": [author.find("{http://www.w3.org/2005/Atom}name").text for author in entry.findall("{http://www.w3.org/2005/Atom}author")],
                "link": entry.find("{http://www.w3.org/2005/Atom}id").text,
            }
            papers.append(paper)
        return papers
    else:
        raise Exception(f"Failed to fetch data from ArXiv API: {response.status_code}")

In [ ]:
@opik.track
def generate_embedding(summary):
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    return embedding_model.encode(summary)

In [ ]:
@opik.track
def add_to_index(title, summary, embedding, faiss_index, external_papers):
    faiss_index.add(np.array([embedding]))
    external_papers.append({"title": title, "summary": summary, "embedding": embedding})

In [20]:
@opik.track
def recommend_with_external_and_uploaded(query_embedding, faiss_index, external_papers, top_k=5):
    distances, indices = faiss_index.search(np.array([query_embedding]), k=top_k)
    recommendations = [{"title": external_papers[i]['title'], "summary": external_papers[i]['summary'] } for i in indices[0]]
    return recommendations

In [21]:
@opik.track(project_name="ResearchLink Recommendation Engine")
def full_pipeline(pdf_path, query, top_k=5):
    """
    Integrates summarization, external data fetching, and recommendations with Anthropic's Claude.
    """
    # Step 1: Summarize uploaded PDF using Claude
    title, document = extract_text_from_pdf(pdf_path)
    # print(document)
    instruction = "Summarize the key findings and contributions of this paper."
    summary = summarize_current_summary(document=document, instruction=instruction, current_summary="")
    embedding = generate_embedding(summary)

    # Initialize FAISS index
    embedding_dim = 384  # Dimension of the embedding model
    faiss_index = faiss.IndexFlatL2(embedding_dim)
    external_papers = []

    # Step 2: Add the summary to the FAISS index
    add_to_index(title, summary, embedding, faiss_index, external_papers)

    # Step 3: Fetch related papers from ArXiv
    fetched_papers = fetch_arxiv_papers(query, max_results=10)
    for paper in fetched_papers:
        paper_embedding = generate_embedding(paper['summary'])
        add_to_index(paper['title'], paper['summary'], paper_embedding, faiss_index, external_papers)

    # Step 4: Generate recommendations
    recommendations = recommend_with_external_and_uploaded(embedding, faiss_index, external_papers, top_k=top_k)

    return title, summary, recommendations

Now, let's enable paper upload from our Colab notebook... Cool? 😎

In [17]:
from google.colab import files
uploaded = files.upload()

Saving 1706.03762v7.pdf to 1706.03762v7.pdf


In [18]:
pdf_path = list(uploaded.keys())[0] # Get the first uploaded file's path
query = "LLM Agents and Transformer Architectures" # Example search query for related papers

Now let's get the `title`, `summary`, and `recommendations` from our `full_pipeline` function.

In [22]:
title, summary, recommendations = full_pipeline(pdf_path, query)

In [23]:
# Display the summary and recommendations
print(f"Summary of Uploaded Paper [{title}]:")
print(summary)

Summary of Uploaded Paper [Provided proper attribution is provided, Google hereby grants permission to]:
## Summary:

This paper introduces a novel neural network architecture, the Transformer, designed for sequence transduction tasks. The Transformer simplifies existing models by relying solely on attention mechanisms, eliminating the need for recurrent or convolutional layers. The authors demonstrate its effectiveness in machine translation, achieving state-of-the-art performance on English-to-German and English-to-French translation tasks. The Transformer outperforms previous models, improving BLEU scores by a significant margin and reducing training time and costs. Additionally, the model's versatility is showcased through successful application in English constituency parsing with varying training data sizes.

## Revised Summary:

The Transformer architecture, a new attention-based model, outperforms traditional sequence transduction models. It achieves 28.4 BLEU on WMT 2014 Engli

Let's check out the recommendations we've got.

In [24]:

print("\nRecommended Papers:")
for rec in recommendations:
    print(f"Title: {rec['title']}")


Recommended Papers:
Title: Provided proper attribution is provided, Google hereby grants permission to
Title: Agent-SiMT: Agent-assisted Simultaneous Machine Translation with Large
  Language Models
Title: Advancing Transformer Architecture in Long-Context Large Language
  Models: A Comprehensive Survey
Title: Towards Collaborative Intelligence: Propagating Intentions and Reasoning
  for Multi-Agent Coordination with Large Language Models
Title: Targeting the Core: A Simple and Effective Method to Attack RAG-based
  Agents via Direct LLM Manipulation


In [25]:
len(recommendations)

5

In [26]:
recommendations

[{'title': 'Provided proper attribution is provided, Google hereby grants permission to',
  'summary': "## Summary:\n\nThis paper introduces a novel neural network architecture, the Transformer, designed for sequence transduction tasks. The Transformer simplifies existing models by relying solely on attention mechanisms, eliminating the need for recurrent or convolutional layers. The authors demonstrate its effectiveness in machine translation, achieving state-of-the-art performance on English-to-German and English-to-French translation tasks. The Transformer outperforms previous models, improving BLEU scores by a significant margin and reducing training time and costs. Additionally, the model's versatility is showcased through successful application in English constituency parsing with varying training data sizes.\n\n## Revised Summary:\n\nThe Transformer architecture, a new attention-based model, outperforms traditional sequence transduction models. It achieves 28.4 BLEU on WMT 2014 

YOHH!!!

And we've got all these tracked and ready for easy and seamless visualization, evaluation and monitoring on the Comet's Opik platform.

The value of Opik is tremendous, I've experimented with various experiment trackers and the likes... Opik from Comet gave me exactly what I envisioned deep down and even more.

I'm still getting to know and apply this amazing tool to my ML/LLM workflow. You could do same too 😉 😁

Happy and joyful Cheers, my friend.

Eddie Otudor